In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [210]:
from sympy import sqrt, cos, sin,  cosh,  sinh, Add
from sympy import Symbol, S, oo, Rational, Function, pi , factorial, O, symbols, lambdify, I
from sympy import Derivative, series, limit, Matrix, simplify, collect, solve, Poly, poly, diff, nsolve, transpose, factor_terms
from sympy import latex, init_printing
from sympy.core.numbers import NegativeOne, One, Zero
from sympy.printing.latex import print_latex

import numpy as np
import pandas as pd
from tqdm import tqdm
import ray
import time

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import defaultdict
from itertools import product, combinations
from functools import reduce
from acchamiltoniansandmatrices.Tracking.LinearMatrixTracking import nestList, GenerateNDimCoordinateGrid
from acchamiltoniansandmatrices.Hamiltonians.LatticeElementHamiltonians import HamDrift6D, HamQuad6D
from acchamiltoniansandmatrices.Hamiltonians.Operators import Poisson
from acchamiltoniansandmatrices.LieMaps.Poisson import PoissonBracket
from acchamiltoniansandmatrices.LieMaps.LieOperator import LieOperator
from acchamiltoniansandmatrices.Utils.Plotting import multi_countour_plot
from acchamiltoniansandmatrices.Factorization.Factorization import truncate, taylorize, dragt_finn_factorization
from acchamiltoniansandmatrices.Utils.Polynomials import getpoly, RotationMatrix2D, explicitCheck, geteigenvects
from acchamiltoniansandmatrices.Matrices.NumpyMatrices import RnpQuad6DChroma
from acchamiltoniansandmatrices.Matrices.SymbMatrices import RsymbQuad6DChroma
from acchamiltoniansandmatrices.Integrators import integrator
from acchamiltoniansandmatrices.Tracking.LinearMatrixTracking import nestList

In [3]:
x, px, y, py, z, delta = symbols('x p_x y p_y z delta', real=True)
eps, h , k0, k1, k2, sigma, betag = symbols('epsilon h k_0 k_1 k_2 sigma beta_gamma', real=True)
L, beta0, gamma0 = symbols(" L beta_0 gamma_0", real=True, positive=True)
x0,px0 = symbols('x_0 px_0') # to not overwrite anything

In [7]:
p = symbols("p")

In [8]:
H1 = p**2+x**2
H2 = p**2

In [63]:
X = Function("X")(x,px)
Y = Function("Y")(x,px)

def bracket_generator(tuplist,X,Y):
    _list = tuplist.copy()
    _list.reverse()
    
    if _list[0][1] > 1 or _list[0][1]==0:
        res = 0
    else:
        # first rightside brackets
        def applybracketX(a):
            return PoissonBracket(X,a,coords=[x],mom=[px])
        
        def applybracketY(a):
            return PoissonBracket(Y,a,coords=[x],mom=[px])

        res = list(nestList(applybracketX,Y,_list[0][0]))[-1]
        
        _list = _list[1:]
        
        while len(_list) >0:
            res = list(nestList(applybracketY,res,_list[0][1]))[-1]
            res = list(nestList(applybracketX,res,_list[0][0]))[-1]
            _list = _list[1:]
    return res


In [72]:
bracket_generator([(1,0),(1,1)],X,Y)

-PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))

In [73]:
def rsdict(n):
    res = defaultdict(set)
    for r in range(0,n+1):
        for s in range(0,n+1):
            res[r+s] = res[r+s].union({(r,s)})
            
    return res

In [ ]:
n = 1
tups = prod

In [74]:
rsdict(3)

defaultdict(set,
            {0: {(0, 0)},
             1: {(0, 1), (1, 0)},
             2: {(0, 2), (1, 1), (2, 0)},
             3: {(0, 3), (1, 2), (2, 1), (3, 0)},
             4: {(1, 3), (2, 2), (3, 1)},
             5: {(2, 3), (3, 2)},
             6: {(3, 3)}})

In [105]:
list([c for c in list(combinations(list(product(range(5),range(5)))[1:],2)) if sum([k+v for k,v in c])  == 5][0])

[(0, 1), (0, 4)]

In [190]:
class bicoloredTree():
    def __init__(self, root):
        self.root = root
        self.children = []
        self.Nodes = []
        
    def addNode(self, node):
        self.children.append(node)
        
    def getAllNodes(self):
        self.Nodes.append(self.root)
        for child in self.children:
            self.Nodes.append(child.color)
        for child in self.children:
            if child.getChildNodes(self.Nodes) != None:
                child.getChildNodes(self.Nodes)
                
    def __mul__(self, other):
        newtree = bicoloredTree(self.root)
        for child in self.children:
            newtree.children.append(child)
        newtree.children.append(other)
        
        return newtree
        
    def __str__(self,level=0):
        ret = "\t"*level+repr(self.root.color)+"\n"
        for child in self.children:
            ret += child.__str__(level+1)
        return ret
            
    
class Node():
    def __init__(self, color):
        self.color = color
        self.children = []
        
    def addNode(self, node):
        self.children.append(node)
        
    def getChildNodes(self, Tree):
        for child in self.children:
            if child.children:
                child.getChildNodes(Tree)
                Tree.append(child.color)
                
            else:
                Tree.append(child.color)
                
    def __str__(self,level=0):
        ret = "\t"*level+repr(self.color)+"\n"
        for child in self.children:
            ret += child.__str__(level+1)
        return ret
        
        
        
        

In [199]:
rootnode = Node('white')
roottree1 = bicoloredTree(rootnode)
roottree1.addNode(Node('black'))
# roottree1.addNode(Node('black'))

roottree2 = bicoloredTree(rootnode)
roottree2.addNode(Node('black'))

roottree3 = roottree1*roottree2

rootnode2 = Node('black')
roottree4 = bicoloredTree(rootnode2)

roottree5 = roottree3*roottree4

In [196]:
print(roottree1)

'white'
	'black'



In [197]:
print(roottree2)

'white'
	'black'



In [200]:
print(roottree5)

'white'
	'black'
	'white'
		'black'
	'black'



In [224]:
dat = pd.read_csv("bchHall20.dat", index_col=None,header=None, delim_whitespace=True,names=["i'","i\"",'p','q'])
dat['coeff'] = dat['p']

,i',"i""",p,q
1,1,0,1,1
2,2,0,1,1
3,2,1,-1,2
4,3,1,1,12
5,3,2,-1,12
...,...,...,...,...
111009,225,223,1103152823,7039647014400
111010,226,223,-67787,4345461120
111011,225,224,-5253124501,33790305669120
111012,226,224,-3304971841,14079294028800


In [229]:
dat.tail()['p'] / dat.tail()['q'].apply(lambda x: int(x))

111009    0.000157
111010   -0.000016
111011   -0.000155
111012   -0.000235
111013   -0.000137
dtype: float64

In [243]:
a=1
b=0
c=2
d=0
ia=1
ib=1

iprime = {1:1,2:2}
idprime = {1:0,2:0}
ia ={1:1,2:1}
ei = {1:X, 2:Y}
si = {1:1,2:1}
ki = {1:0,2:0}

roottreeblack = bicoloredTree(Node('black'))
roottreewhite = bicoloredTree(Node('white'))
ui = {1: roottreeblack, 2:roottreewhite}
i=3
for n in range(2,10):
    for j in range(1,i):
        for k in range(j+1,i):
            if ia[j] + ia[k] == n and j >= idprime[k]:
                idprime[i] = j
                iprime[i] =k
                ia[i]=n
                
                # ki
                if idprime[iprime[i]] != idprime[i]:
                    ki[i] = 1
                else:
                    ki[i] = ki[iprime[i]] + 1
                si[i] = ki[i]*si[iprime[i]]*si[idprime[i]]
                ui[i] = ui[iprime[i]] * ui[idprime[i]]
                ei[i]=PoissonBracket(ei[iprime[i]],ei[idprime[i]],coords=[x],mom=[px])
                print("i:{:2} , |i|:{:2}, i':{:2}, i\":{:2}, s_i:{:2} ".format(i,ia[i],iprime[i],idprime[i],si[i],display(ei[i])))
                i = i+1


-PoissonBracket(X(x, p_x), Y(x, p_x))

i: 3 , |i|: 2, i': 2, i": 1, s_i: 1 


PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))

i: 4 , |i|: 3, i': 3, i": 1, s_i: 2 


PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))

...

i:125 , |i|: 9, i':12, i": 8, s_i:12 


PoissonBracket(-PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))), PoissonBracket(PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x)), Y(x, p_x)))

i:126 , |i|: 9, i':13, i": 8, s_i: 4 


PoissonBracket(-PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))), PoissonBracket(PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x)), Y(x, p_x)))

i:127 , |i|: 9, i':14, i": 8, s_i: 2 


In [234]:
ia

{1: 1,
 2: 1,
 3: 2,
 4: 3,
 5: 3,
 6: 4,
 7: 4,
 8: 4,
 9: 5,
 10: 5,
 11: 5,
 12: 5,
 13: 5,
 14: 5,
 15: 6,
 16: 6,
 17: 6,
 18: 6,
 19: 6,
 20: 6,
 21: 6,
 22: 6,
 23: 6}

In [266]:
res = X + Y

for i in range(1,3):
    # get the relevant brackets
    ilist = [k for k,v in ia.items() if v==i]
    bracketlist = [ei[j] for j in ilist]
    
    # get the coeffcients
    coefflist = [Rational(dat.loc[j,'p'],dat.loc[j,'q']) for j in ilist]
    print(coefflist)
    print(bracketlist)
    for c,b in zip(coefflist, bracketlist):
        res += c * b
res

[1, 1]
[X(x, p_x), Y(x, p_x)]
[-1/2]
[-PoissonBracket(X(x, p_x), Y(x, p_x))]


2*X(x, p_x) + 2*Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2

In [242]:
#e**X*e**Y
res.subs(X,px**2).subs(Y,px**2+x**2).doit()

98*p_x**2/45 - 10*p_x*x/3 + 107*x**2/45

In [208]:
for i in range(3,6):
    print(ui[i])

'white'
	'black'

'white'
	'black'
	'black'

'white'
	'black'
	'white'



In [245]:
from acchamiltoniansandmatrices.LieMaps.BCH import generate_bch_expression

In [263]:
generate_bch_expression(order=5,coords=[x],mom=[px])

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 + PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 - PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12 + PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)))/120 + PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x)))/360 + PoissonBracket(PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x))/24 - PoissonBracket(PoissonBracket(PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x)), X(x, p_x))/720 - PoissonBracket(PoissonBracket(PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x)), Y(x, p_x))/180 + PoissonBracket(PoissonBracket(PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x)), Y(x, p_x))/180 + PoissonBracket(PoissonBracket(PoissonBracket(-PoissonBracket(X(x, p_x),

In [137]:
display(ei[4])

PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))

In [132]:
ei[3]

-PoissonBracket(X(x, p_x), Y(x, p_x))

In [133]:
ei[4]

PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))

In [134]:
ei[5]

PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))

In [135]:
ei[6]

PoissonBracket(PoissonBracket(-PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x))

In [17]:
def rsdict(n):
    res = defaultdict(set)
    for r in range(0,n+1):
        for s in range(0,n+1):
            res[r+s] = res[r+s].union({(r,s)})
            
    return res

def rscoeff(rsdict,n):
    rssum  = dict()
    rsprod = dict()
    for i in range(2,n):
            rssum[i]  = sum([r+s for r,s in rsdict[i]])
            rsprod[i] = reduce((lambda k,l: k *l),)

In [18]:
rsdict(2)

defaultdict(set, {2: {(1, 1)}, 3: {(1, 2), (2, 1)}, 4: {(2, 2)}})

# Effective Hams 